In [9]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [10]:
splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
df_train_5000_with_embeddings = pd.read_pickle("train_5000_with_embeddings.pkl")
df_test = pd.read_parquet("hf://datasets/wangrongsheng/ag_news/" + splits["test"])
df_test_1000 = df_test.sample(n=1000, random_state=42)

from sentence_transformers import SentenceTransformer

def encode_text(text):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    return model.encode(text)

from sklearn.metrics.pairwise import cosine_similarity

def select_top_k(query_embedding, candidate_embeddings, k):
    similarities = cosine_similarity([query_embedding], candidate_embeddings)[0]
    top_k_indices = np.argsort(similarities)[-k:][::-1]
    return top_k_indices

In [11]:
df_train_5000_with_embeddings.iloc[0]['text']

'BBC set for major shake-up, claims newspaper London - The British Broadcasting Corporation, the world #39;s biggest public broadcaster, is to cut almost a quarter of its 28 000-strong workforce, in the biggest shake-up in its 82-year history, The Times newspaper in London said on Monday.'

In [19]:
from transformers import pipeline
import torch

input_text = "What is the topic of the input? World, sports, business or technology?\nThe response should follow the format: Topic:{world, sports, business or technology}\nReason:{reason}" + f"\nInput: {df_train_5000_with_embeddings.iloc[0]['text']}" + "\nLet's think step by step."

messages = [
    {"role": "user", "content": input_text},
]
pipe = pipeline("text-generation", 
                model="meta-llama/Llama-3.2-3B-Instruct",
                device_map="auto"
               )
pipe(messages)[0]['generated_text'][1]['content']

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

'Topic: **technology** \n\nReason: The input discusses a major shake-up at the BBC'

In [12]:
def zero_shot_cot_LLAMA(input_, model, tokenizer):
    messages = [
        {"role": "user", "content": "What is the topic of the input? World, sports, business or technology?\nThe response should follow the format: Topic:{world, sports, business or technology}\nReason:{reason}" + f"\nInput: {input_}" + "\nLet's think step by step."},
    ]
    pipe = pipeline("text-generation", 
                    model=model,
                    tokenizer=tokenizer,
                    pad_token_id=tokenizer.eos_token_id
                   )
    completion = pipe(messages)
    return completion[0]['generated_text'][1]['content']

def few_shot_cot_LLAMA(examples, input_, model, tokenizer):
    messages = [
        {"role": "user", "content": "What is the topic of the input? World, sports, business or technology?" + f"\n{examples}" +"\nThe response should follow the format: Topic:{world, sports, business or technology}\nReason:{reason}\nHere is the test data" + f"\nInput: {input_}" + "\nLet's think step by step."},
    ]
    pipe = pipeline("text-generation", 
                    model=model,
                    tokenizer=tokenizer,
                    pad_token_id=tokenizer.eos_token_id
                   )
    completion = pipe(messages)
    return completion[0]['generated_text'][1]['content']

In [14]:
from collections import Counter
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils import logging
logging.set_verbosity_error()

def iterative_demonstration_selection(test_sample, train_samples, model_id, k=4, q=2):

    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda:0")

    label_dict = {0: "World", 1: "Sports", 2: "Business", 3: "Technology"}
    train_embeddings = np.stack(train_samples['embedding'].to_numpy())
    all_answers = []
    reasoning_path = zero_shot_cot_LLAMA(test_sample, model, tokenizer)

    for _ in range(q):
        query_embedding = encode_text(reasoning_path)
        selected_indices = select_top_k(query_embedding, train_embeddings, k)
        demonstrations = [train_samples.iloc[i] for i in selected_indices]

        examples_prompt = "\n".join(
            [f"Input: {row['text']}\nTopic: {label_dict[row['label']]}" for row in demonstrations]
        )

        result = few_shot_cot_LLAMA(examples_prompt, test_sample, model, tokenizer)

        try:
            topic_line = next(line for line in result.split('\n') if line.lower().startswith("topic"))
            reason_line = next(line for line in result.split('\n') if line.lower().startswith("reason"))
        except StopIteration:
            topic_line = "Topic: unknown"
            reason_line = "Reason: unknown"

        reasoning_path = reason_line
        answer = topic_line.split(":", 1)[-1].strip().lower()
        all_answers.append(answer)

    final_answer = Counter(all_answers).most_common(1)[0][0]
    return final_answer

In [17]:
def run_ids_on_test_set(df_test, df_train, model_id, k=4, q=3):
    predictions = []
    reasoning_paths = []

    for i, row in tqdm(df_test.iterrows(), total=len(df_test), desc="Running IDS on test set"):
        test_text = row['text']
        true_label = row['label']
        
        # Run IDS
        try:
            final_answer = iterative_demonstration_selection(test_text, df_train, k=k, q=q, model_id=model_id)
        except Exception as e:
            final_answer = "error"
            print(f"Error on test sample {i}: {e}")

        predictions.append(final_answer)
        reasoning_paths.append(test_text)

    df_test_result = df_test.copy()
    df_test_result['prediction'] = predictions
    df_test_result['input'] = reasoning_paths

    return df_test_result

In [8]:
from sklearn.metrics import accuracy_score

df_test_result = run_ids_on_test_set(df_test_1000, df_train_5000_with_embeddings, k=4, q=3, model_id="meta-llama/Llama-3.2-3B-Instruct")

Running IDS on test set:   0%|          | 0/1000 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

Running IDS on test set: 100%|█████████▉| 998/1000 [2:55:52<00:19,  9.78s/it]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Running IDS on test set: 100%|█████████▉| 999/1000 [2:56:01<00:09,  9.63s/it]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Running IDS on test set: 100%|██████████| 1000/1000 [2:56:11<00:00, 10.57s/it]


In [9]:
label_dict = {0: "World", 1: "Sports", 2: "Business", 3: "Technology"}
df_test_result['label_str'] = df_test_result['label'].map(label_dict).str.lower()
accuracy = accuracy_score(df_test_result['label_str'], df_test_result['prediction'])
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 86.30%


In [10]:
df_test_result[['input','label_str','prediction']].to_csv("agnews_ids_results_LLAMA.csv", index=False)

## Gemma 2b it

In [7]:
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="google/gemma-2-2b-it")
pipe(messages)

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Device set to use cuda:0


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': "I am Gemma, an open-weights AI assistant. I'm a large language model, trained"}]}]

In [20]:
df_test_result = run_ids_on_test_set(df_test_1000, df_train_5000_with_embeddings, k=4, q=3, model_id="google/gemma-2-2b-it")

Running IDS on test set:   0%|          | 0/1000 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Running IDS on test set:   0%|          | 1/1000 [00:09<2:37:56,  9.49s/it]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Running IDS on test set:   0%|          | 2/1000 [00:19<2:39:08,  9.57s/it]

...

Running IDS on test set: 100%|█████████▉| 998/1000 [2:54:30<00:21, 10.61s/it]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Running IDS on test set: 100%|█████████▉| 999/1000 [2:54:40<00:10, 10.44s/it]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Running IDS on test set: 100%|██████████| 1000/1000 [2:54:51<00:00, 10.49s/it]


In [23]:
df_test_result['prediction'].unique()

array(['**sports**', '**technology**', '**business**', '**world**',
       'business', 'sports', 'unknown', '{world}', 'technology'],
      dtype=object)

In [24]:
def clean_prediction(pred):
    pred = pred.lower().strip()
    pred = pred.replace("*", "").replace("{", "").replace("}", "").strip()
    if pred not in ["world", "sports", "business", "technology"]:
        return "unknown"
    return pred
df_test_result['prediction_clean'] = df_test_result['prediction'].apply(clean_prediction)

In [28]:
df_test_result

,text,label,prediction,input,prediction_clean,label_str
7094,Fan v Fan: Manchester City-Tottenham Hotspur T...,1,**sports**,Fan v Fan: Manchester City-Tottenham Hotspur T...,sports,sports
1017,Paris Tourists Search for Key to 'Da Vinci Cod...,0,**technology**,Paris Tourists Search for Key to 'Da Vinci Cod...,technology,world
2850,Net firms: Don't tax VoIP The Spanish-American...,3,**business**,Net firms: Don't tax VoIP The Spanish-American...,business,technology
1452,Dependent species risk extinction The global e...,3,**technology**,Dependent species risk extinction The global e...,technology,technology
457,EDS Is Charter Member of Siebel BPO Alliance (...,3,**business**,EDS Is Charter Member of Siebel BPO Alliance (...,business,technology
...,...,...,...,...,...,...
4127,Britain Charges Cleric Sought by US for Aiding...,0,**world**,Britain Charges Cleric Sought by US for Aiding...,world,world
4801,Hobbit-sized Humans Called Homo floresiensis D...,3,**technology**,Hobbit-sized Humans Called Homo floresiensis D...,technology,technology
4919,Tar Heels beat Miami For the second time this ...,1,**sports**,Tar Heels beat Miami For the second time this ...,sports,sports
1721,Microsoft Eyes Video for Business IM Software ...,3,**technology**,Microsoft Eyes Video for Business IM Software ...,technology,technology


In [30]:
from sklearn.metrics import accuracy_score
label_dict = {0: "World", 1: "Sports", 2: "Business", 3: "Technology"}
df_test_result['label_str'] = df_test_result['label'].map(label_dict).str.lower()
accuracy = accuracy_score(df_test_result['label_str'], df_test_result['prediction_clean'])
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 85.80%


In [31]:
df_test_result[['input','label_str','prediction_clean']].to_csv("agnews_ids_results_gemma.csv", index=False)